Using Spacy Neuralcoref to detect coreference chains involving 'La Defense' and find its designations

In [ ]:
!pip install spacy==2.1.0 
!python3 -m spacy download en_core_web_lg 
!pip install neuralcoref --no-binary neuralcoref


     |████████████████████████████████| 27.7MB 101kB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 2.1MB 45.0MB/s 
     |████████████████████████████████| 3.2MB 38.2MB/s 
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: preshed 3.0.5
    Uninstalling preshed-3.0.5:
      Successfully uninstalled preshed-3.0.5
  Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 826.9MB 1.2MB/s 
  Created wheel for en-core-web

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)



Mounted at /content/drive


In [ ]:
import re

from bs4 import BeautifulSoup
import pandas as pd

import spacy

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, classification_report

import neuralcoref

#Chargement Corpus

In [ ]:
chemin = "/content/drive/MyDrive/Stage Defense"
fichierxml = chemin + "/Corpus/Presse/grandfinalEN_annot_partiel.xml"
cheminmeronymes = chemin + "/Ressources/Liste_meronymes.txt"


In [ ]:
nlp = spacy.load('en_core_web_lg')

Designations detection

In [ ]:
neuralcoref.add_to_pipe(nlp, blacklist=False )

In [ ]:
def Main_coref(fichierxml, fichierxmlsortie, eval_articles = [], dico_entites = {}, fin=None):
  soup = BeautifulSoup(open(fichierxml, encoding="utf8"),"lxml")
  y_true, y_pred = [], []
  for element in soup.find_all(["titre","texte"]):
    article = element.parent
    tag = element.name
    id = int(article.get("id"))
    if fin and id > fin : break
    print("article numéro ",id)
    if id in eval_articles:
      eval = True
    else: eval = False

    nouveautexte , y_true_part, y_pred_part = detecte_mentions_coref(element,eval=eval, entites_cat=dico_entites)
    
    nouvel_elem = BeautifulSoup(f"<{tag}>{nouveautexte}</{tag}>", 'html.parser')
    element.replaceWith(nouvel_elem)
    
    if eval:
      y_true.extend(y_true_part)
      y_pred.extend(y_pred_part)


  open(fichierxmlsortie,'w',encoding="utf8").write(str(soup))
  print("enregistré dans", fichierxmlsortie)
  
  if eval_articles:

    return affiche_resultats(y_true,y_pred)



    

In [ ]:
def extrait_indices(element):
  listementions = []
  txt = element.text
  for mention in element.find_all(["mention","meronyme"]):
    listementions.append([mention.text,mention.name])
    mention.string = "MENTION_COUPURE"
  
  coupes = element.text.split("MENTION_COUPURE")
  fin = 0
  for i in range(len(listementions)):
    debut = fin
    debut += len(coupes[i]) 

    fin = debut + len(listementions[i][0])
    listementions[i].append((debut,fin))
    #print(i,listementions[i], txt[debut:fin])

  return listementions



In [ ]:
def classifie_tokens(doc,liste_mentions):
  tokens_cat = []
  for token in doc:
    debut_token = token.idx
    fin_token = debut_token + len(token.text)
    cat = "O"
    for mention in liste_mentions:
      debut_men, fin_men = mention[2]
      if debut_men <= debut_token and fin_men >= fin_token : 
        cat = mention[1]
    tokens_cat.append((token.text,cat))
  return tokens_cat
  


In [ ]:
def affiche_resultats(y_true,y_pred):
  labels = ["O","mention","meronyme"]
  assert len(y_true)== len(y_pred)
  df = pd.DataFrame(y_true, columns = ["token", "true"])
  df2 = pd.DataFrame(y_pred, columns = ["token", "pred"])
  df["pred"] = df2.pred
  print(df.head(60))
  y_true = df.true.to_numpy()
  y_pred = df.pred.to_numpy()
  report = classification_report(y_true,y_pred)
  print(report)
  return report, df
  rappel = recall_score(y_true,y_pred,labels, average=None)
  precision = precision_score(y_true,y_pred,labels, average=None)
  df_results = pd.DataFrame(list(zip(rappel,precision)), columns=["rappel","precision"], index=labels)
  return df_results, df




In [ ]:
def is_chevauchement(span, liste_mentions):
  debut_NC = span.start_char
  fin_NC = span.end_char
  chevauchement = False
  for men in liste_mentions:
    debut_men, fin_men = men[2]
    if debut_men <= debut_NC <= fin_men or debut_NC <= debut_men <= fin_NC :
      chevauchement = True
      break
  return chevauchement


In [ ]:
def detecte_mentions_coref(element,entites_cat={"mention":[ "paris la defense","la defense"]},eval=False):
  entites_cat = {ent:cat for cat in entites_cat for ent in entites_cat[cat]}
  
  texte = element.text
  doc = nlp(texte)
  y_pred_part,y_true_part = [], []
  if eval:
    liste_mentions_true = extrait_indices(element)
    y_true_part = classifie_tokens(doc,liste_mentions_true)

  liste_mentions_pred = []
  for cluster in doc._.coref_clusters:
    entite_main = normalise(cluster.main.text)
    
    
    for entite in entites_cat:
    
      #if entite in entite_main:
      if re.match(".*?\\b"+entite+"\\b", entite_main) :
      
        for mention in cluster.mentions:
          if not is_chevauchement(mention,liste_mentions_pred):
            liste_mentions_pred.append([mention.text,entites_cat[entite],(mention.start_char,mention.end_char)])
        break

  GNs = sorted(list(doc.noun_chunks)+list(doc.ents), key = lambda x: x.start_char)
  for NC in GNs:
    regstring = normalise(NC.text)
    for entite in entites_cat:
      
      #if entite in regstring:
      if re.match(".*?\\b"+entite+"\\b" , regstring):
        debut_NC = NC.start_char
        fin_NC = NC.end_char
        chevauchement = is_chevauchement(NC, liste_mentions_pred)

        if not chevauchement:
          liste_mentions_pred.append([NC.text,entites_cat[entite], (debut_NC,fin_NC)])
          break

  liste_mentions_pred = sorted(liste_mentions_pred, key=lambda x:x[2][0])
  y_pred_part = classifie_tokens(doc,liste_mentions_pred)
  texte = ajoute_balise(doc.text,liste_mentions_pred)
  
  return texte,y_true_part,y_pred_part



In [ ]:
def ajoute_balise(texte, listementions):
  texte = list(texte)
  for mention in reversed(listementions):
    
    tag = mention[1]
    mentxt = mention[0]
    debut,fin = mention[2]
    texte[debut:fin] =  list(f"<{tag}>{mentxt}</{tag}>")
  

  return "".join(texte)

In [ ]:
import unicodedata

def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)


def normalise(text):
  text = strip_accents(text.lower())
  text =  re.sub("[\(\)\[\]\n\*\?\+]","",text)
  return text

In [ ]:
blacklist = ["Being","Sense"]

In [ ]:
sortie = chemin+ "/Corpus/annotation_coref_spacy_en.xml"

In [ ]:

dico_entites = {"mention":["la defense","district"],"meronyme": [normalise(mero) for mero in open(cheminmeronymes, encoding="utf8") if mero.strip("\n") not in blacklist]}

In [ ]:
#dico_entites = {"mention":["la defense","district"]}

In [ ]:
resultats,token_comp = Main_coref(fichierxml=fichierxml,fichierxmlsortie=sortie, eval_articles=list(range(354,365)), dico_entites=dico_entites, fin=None)

article numéro  354
article numéro  354
article numéro  355
article numéro  355
article numéro  356
article numéro  356
article numéro  357
article numéro  357
article numéro  358
article numéro  358
article numéro  359
article numéro  359
article numéro  360
article numéro  360
article numéro  361
article numéro  361
article numéro  362
article numéro  362
article numéro  363
article numéro  363
article numéro  364
article numéro  364
article numéro  365
article numéro  365
article numéro  366
article numéro  366
article numéro  367
article numéro  367
article numéro  368
article numéro  368
article numéro  369
article numéro  369
article numéro  370
article numéro  370
article numéro  371
article numéro  371
article numéro  372
article numéro  372
article numéro  373
article numéro  373
article numéro  374
article numéro  374
article numéro  375
article numéro  375
article numéro  376
article numéro  376
article numéro  377
article numéro  377
article numéro  378
article numéro  378


texte1

In [ ]:
print(resultats)


              precision    recall  f1-score   support

           O       0.99      0.99      0.99     11071
     mention       0.47      0.84      0.60       119
    meronyme       0.28      0.23      0.25       108

    accuracy                           0.98     11298
   macro avg       0.58      0.69      0.61     11298
weighted avg       0.98      0.98      0.98     11298

